###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [2]:
import pandas as pd

###Archivo ec2r-4byk.csv

Contiene información sobre la afectación a la infraestructura crítica: voladura de oleoductos. Se mide en casos.

In [3]:
df = pd.read_csv("https://www.datos.gov.co/resource/ec2r-4byk.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad
0,2024-01-23T00:00:00.000,81,ARAUCA,81065,ARAUQUITA,1
1,2024-01-04T00:00:00.000,81,ARAUCA,81065,ARAUQUITA,1
2,2023-10-21T00:00:00.000,86,PUTUMAYO,86320,ORITO,1
3,2023-10-09T00:00:00.000,52,NARIÑO,52573,PUERRES,1
4,2023-10-08T00:00:00.000,86,PUTUMAYO,86320,ORITO,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292 entries, 0 to 1291
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   fecha_hecho   1292 non-null   object
 1   cod_depto     1292 non-null   int64 
 2   departamento  1292 non-null   object
 3   cod_muni      1292 non-null   object
 4   municipio     1292 non-null   object
 5   cantidad      1292 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 60.7+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [5]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad']
blowingup_of_pipelines_MinDefensa = df[relevant_cols]
blowingup_of_pipelines_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292 entries, 0 to 1291
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_muni      1292 non-null   object
 1   departamento  1292 non-null   object
 2   municipio     1292 non-null   object
 3   fecha_hecho   1292 non-null   object
 4   cantidad      1292 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 50.6+ KB


### Convertir columna fecha_hecho a tipo date

In [6]:
# Hacer una copia explícita del DataFrame
blowingup_of_pipelines_MinDefensa_ = blowingup_of_pipelines_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
blowingup_of_pipelines_MinDefensa_['fecha_hecho'] = pd.to_datetime(blowingup_of_pipelines_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
blowingup_of_pipelines_MinDefensa_['fecha_hecho'] = blowingup_of_pipelines_MinDefensa_['fecha_hecho'].dt.year

In [7]:
blowingup_of_pipelines_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010, 2009, 2008, 2007], dtype=int32)

### Verificar valores nulos

In [8]:
blowingup_of_pipelines_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
fecha_hecho,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [9]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(blowingup_of_pipelines_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ARAUCA' 'PUTUMAYO' 'NARIÑO' 'NORTE DE SANTANDER' 'BOYACA' 'CESAR'
 'SANTANDER' 'CASANARE' 'ANTIOQUIA' 'HUILA' 'LA GUAJIRA' 'TOLIMA'
 'VALLE DEL CAUCA' 'BOLIVAR']

Categorías en la columna 'municipio':
['ARAUQUITA' 'ORITO' 'PUERRES' 'CUCUTA' 'TEORAMA' 'SARAVENA' 'MALLAMA'
 'PUPIALES' 'GUACHUCAL' 'CUBARA' 'TIBU' 'EL CARMEN' 'TAMALAMEQUE'
 'RICAURTE' 'CURUMANI' 'EL TARRA' 'BARRANCABERMEJA' 'RIONEGRO' 'TOLEDO'
 'BARBACOAS' 'CONVENCION' 'ARAUCA' 'ALDANA' 'SARDINATA' 'AGUACHICA'
 'SAN ANDRES DE TUMACO' 'VALLE DEL GUAMUEZ' 'PUERTO ASIS' 'AGUAZUL'
 'LA GLORIA' 'CHIMICHAGUA' 'FORTUL' 'SAN MIGUEL' 'IPIALES' 'YONDO' 'NEIVA'
 'CONTADERO' 'PUERTO CAICEDO' 'CORDOBA' 'SAN JUAN DEL CESAR' 'CHAPARRAL'
 'TAME' 'GIGANTE' 'SANTO DOMINGO' 'ZARAGOZA' 'TRINIDAD' 'VILLAGARZON'
 'MOCOA' 'BARRANCAS' 'BUENAVENTURA' 'MAICAO' 'CANTAGALLO' 'SAN PEDRO'
 'AIPE' 'EL MOLINO' 'ORTEGA']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [10]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [11]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    blowingup_of_pipelines_MinDefensa_[col] = blowingup_of_pipelines_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [12]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df blowingup_of_pipelines_MinDefensa_

In [13]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
blowingup_of_pipelines_MinDefensa_['cod_muni'] = blowingup_of_pipelines_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = blowingup_of_pipelines_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [14]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [15]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,1292,1.0


In [16]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = blowingup_of_pipelines_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    81065
1    81065
2    86320
3    52573
4    86320



Nota: el 100%  de los codigos que identifican a los municipios en el df blowingup_of_pipelines_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [17]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [18]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [19]:
# Comparar listas de departamento
compare_lists(blowingup_of_pipelines_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en blowingup_of_pipelines_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en blowingup_of_pipelines_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [20]:
# Comparar listas de códigos
compare_lists(blowingup_of_pipelines_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en blowingup_of_pipelines_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en blowingup_of_pipelines_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [21]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_blowingup_of_pipelines_MinDefensa = blowingup_of_pipelines_MinDefensa_.drop(columns=columns_to_drop)

In [23]:
# Adicionar columna para trazabilidad de la fuente
final_blowingup_of_pipelines_MinDefensa['source_id'] = 27

In [24]:
final_blowingup_of_pipelines_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'source_id'], dtype='object')

In [25]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_blowingup_of_pipelines_MinDefensa.rename(columns=translation_map, inplace=True)

In [26]:
#Estructura final del dataset a integrar a la base de datos
final_blowingup_of_pipelines_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292 entries, 0 to 1291
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dane_code         1292 non-null   object
 1   year_of_incident  1292 non-null   int32 
 2   amount            1292 non-null   int64 
 3   source_id         1292 non-null   int64 
dtypes: int32(1), int64(2), object(1)
memory usage: 35.5+ KB


## Salvar en archivo csv en el drive

In [27]:
# Guardar en archivos CSV en el drive
final_blowingup_of_pipelines_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/blowingup_of_pipelines_MinDefensa.csv', index=False)